<figure>
<img src="../Imagenes/logo-final-ap.png"  width="80" height="80" align="left"/> 
</figure>

# <span style="color:blue"><left>Aprendizaje Profundo</left></span>

# <span style="color:red"><center>Hugging Face: Tokenizadores y datasets</center></span>

<center>T5, Longformers, ST-Moe32B, PaLM </center>

##   <span style="color:blue">Profesores</span>

### Coordinador


- Campo Elías Pardo Turriago, cepardot@unal.edu.co 

### Conferencistas


- Alvaro  Montenegro, PhD, ammontenegrod@unal.edu.co
- Daniel  Montenegro, Msc, dextronomo@gmail.com 
- Oleg Jarma, Estadístico, ojarmam@unal.edu.co 

## <span style="color:blue">Asesora Medios y Marketing digital</span>

- Maria del Pilar Montenegro, pmontenegro88@gmail.com 



## <span style="color:blue">Asistentes</span>



- Nayibe Yesenia Arias, naariasc@unal.edu.co
- Venus Celeste Puertas, vpuertasg@unal.edu.co 

## <span style="color:blue">Referencias</span>

1. [What is Tokenization in NLP? Here’s All You Need To Know](https://www.analyticsvidhya.com/blog/2020/05/what-is-tokenization-nlp/#:~:text=Hence%2C%20tokenization%20can%20be%20broadly,n%2Dgram%20characters%20tokenization.)
1.[Byte Pair Encoding — The Dark Horse of Modern NLP](https://towardsdatascience.com/byte-pair-encoding-the-dark-horse-of-modern-nlp-eb36c7df4f10)
1. [Aprendizaje Profundo-Diplomado](https://github.com/AprendizajeProfundo/Diplomado)
1. [WordPiece: Subword-based tokenization algorithm](https://towardsdatascience.com/wordpiece-subword-based-tokenization-algorithm-1fbd14394ed7)
1. [SentencePiece Tokenizer Demystified](https://towardsdatascience.com/sentencepiece-tokenizer-demystified-d0a3aac19b15)
1. [Aprendizaje Profundo-PLN](https://github.com/AprendizajeProfundo/PLN)
1. [Ashish Vaswani et al.,   Attention Is All You Need](https://arxiv.org/pdf/1706.03762.pdf), diciembre 2017.
1. [Dennis Rothman, Transformers for Natural Language processing](http://libgen.rs/search.php?req=Transformers+for+Natural+Language+processing&open=0&res=25&view=simple&phrase=1&column=def), enero 2021.
1.[ Varios,  Dive into deep learning](https://d2l.ai/), enero 2021

## <span style="color:blue">Contenido</span>

* [Introducción](#Introducción)
* [Tipos de Tokenización Moderna](#Tipos-de-Tokenización-Moderna)
    * [BPE](#BPE)
    * [WordPiece](#WordPiece)
    * [Unigram](#Unigram)
    * [SentencePiece](#SentencePiece)
* [Datasets](#SUPERGLUE-Score)

## <span style="color:blue">Introducción</span>

Como hemos visto en lecciones anteriores, la tokenización juega un papel fundamental cuando se trabaja en NLP o NLU.

Recordemos que tokenizar signfica segmentar el texto en partes fundamentales (**tokens**).

*Los Tokens son los bloques de construcción del lenguaje natural.*

**¿Qué signfica una parte fundamental?**

La respuesta a esta pregunta puede variar:

- Palabras (Word Tokenization)
- Caracteres (Character Tokenization)
- Subpalabras (Subword Tokenization)

Cada una de las tres tiene sus ventajas y sus desventajas.

Por un lado, una tokenización por palabras puede guardar información relevante semántica del texto que se está analizando, pero tenemos el problema de tener vocabularios con un grande e imanegable número de tokens.

Note por ejemplo, la palabra *jugar*. 

Algunas variantes son: *jugué, jugaré, jugaremos, juguemos, ...*.

Esto hace difícil el manejo de textos, pues si en textos nuevos que vea un modelo de NLP aparecen estas variantes y no se encuentran en el vocabulario con el que fue entrenado el modelo, se considerarán **OOV** (Out Of Vocabulary) y se perderá información relevante al momento de hacer predicciones o clasificación.

Como ejemplo, recordemos que tokenizar una frase por palabras puede ser de la siguiente manera:

In [39]:
import re
#String
texto = 'Los pajaritos vuelan, y las nubes se enlazan con un azul infinito.'
# Separar puntos y comas
texto = texto.translate(str.maketrans({'.': ' .', ',': ' ,'}))
# Dividir por espacios
tokens = texto.split()
# Ver tokenización
tokens

['Los',
 'pajaritos',
 'vuelan',
 ',',
 'y',
 'las',
 'nubes',
 'se',
 'enlazan',
 'con',
 'un',
 'azul',
 'infinito',
 '.']

Como vemos, si un modelo se entrenara con esta única frase y de repente le entraramos la palabra *nubes*, el modelo no podría indentificar dicho token como relacionado con *nube* y le daría una etiqueta especial **UNK** (Unknown).

Esto no es óptimo. Sin embargo, este tipo de tokenización se utiliza en **Word2Vec** y **GloVe**.

Por otro lado, la tokenización por caracteres, si bien resuelve este problema de **OOV**, los tokens individuales carecen de significado por si sólo, lo que haría que el modelo pasara mal rato tratando de hacer tareas complicadas del lenguaje, como por ejemplo ahcer inferencias a partir de una frase. 

Además, la longitud de las secuencias de input del modelo se volverán rápidamente muy extensas, lo que haría que el modelo tuviese que hacer una gran cantidad (en algunos casos imposible) de cálculos.

Recordemos cómo tokenizar por caracteres:

In [ ]:
c_tokens = [t for t in texto]
for c in c_tokens:
    print(c,end='|')

L|o|s| |p|a|j|a|r|i|t|o|s| |v|u|e|l|a|n| |,| |y| |l|a|s| |n|u|b|e|s| |s|e| |e|n|l|a|z|a|n| |c|o|n| |u|n| |a|z|u|l| |i|n|f|i|n|i|t|o| |.|

Ante este tipo de dificultades, podríamos preguntarnos qué pasaría en un mundo intermedio, donde no nos quedemos con las palabras completas, pero tampoco simplifiquemos hasta el mundo de caracteres.

Bienvenidos a la tokenización por subpalabras.

En particular, el uso de n-gramas fue potenciado por Mikolov T. en su desarrollo de **FastText** en [Enriching Word Vectors with Subword Information](https://arxiv.org/pdf/1607.04606.pdf).

Veamos a profundidad tipos de tokenización utilizados en los modelos modernos (Transformers).

## <span style="color:blue">Tipos de Tokenización Moderna</span>

El fin último de la tokenización es la construcción del vocabulario que usará el modelo para convertir texto en números.

Un vocabulario eficiente no preservará toda la información de los tokens, sino los más frecuentes según algún umbral.

Una tokenización por subpalabras, se puede entender de la siguiente manera:

- Jugaremos -> Jugar-emos
- Jugarás   -> Jugar-ás

Como podemos notar, esto puede reducir en gran parte el tamaño del vocabulario a la vez que se preservan significados de palabras en su forma básica.

Todos los modelos modernos de Transformers utlizan alguna forma de tokenización por subpalabras.

Veamos algunos de ellos:

#### <span style="color:blue">BPE</span>

**BPE** viene de Byte-Pair Encoding, y se encarga de segmentar palabras **OOV** como subpalabras, y representar dichas palabras con tokens de las subpalabras.

El procedimiento de BPE es como sigue:

1. Divide las palabras del corpus en caracteres después de agregar \</w\>.
1. Inicializar el vocabulario con caracteres únicos en el corpus.
1. Calcule la frecuencia de un par de caracteres o secuencias de caracteres en corpus.
1. Combinar el par más frecuente en el corpus.
1. Guarda la mejor pareja para el vocabulario.
1. Repita los pasos 3 a 5 para un cierto número de iteraciones.

Entendamos los pasos anteriores con un ejemplo:

||Corpus||
|-|-|-|
|low|lower|newest|
|low|lower|newest|
|low|widest|newest|
|low|widest|newest|
|low|widest|newest|

1. Divide las palabras del corpus en caracteres después de agregar \</w\>:

||Corpus||
|-|-|-|
|low\</w\>|lower\</w\>|newest\</w\>|
|low\</w\>|lower\</w\>|newest\</w\>|
|low\</w\>|widest\</w\>|newest\</w\>|
|low\</w\>|widest\</w\>|newest\</w\>|
|low\</w\>|widest\</w\>|newest\</w\>|

||Corpus||
|-|-|-|
|l o w \</w\>|l o w e r \</w\>|n e w e s t \</w\>|
|l o w \</w\>|l o w e r\</w\>|n e w e s t \</w\>|
|l o w \</w\>|w i d e s t \</w\>|n e w e s t \</w\>|
|l o w \</w\>|w i d e s t \</w\>|n e w e s t \</w\>|
|l o w \</w\>|w i d e s t \</w\>|n e w e s t \</w\>|

2. Inicializar el vocabulario con caracteres únicos en el corpus.

|||||||||||
|-|-|-|-|-|-|-|-|-|-|
|d|e|i|l|n|o|r|s|t|w|
|-|-|-|-|-|-|-|-|-|-|

**Iteración 1:**

3. Calcule la frecuencia de un par de caracteres o secuencias de caracteres en corpus.

||||
|-|-|-|
|d-e (3)|l-o (7)|t-\</w\> (8)|
|e-r (2)|n-e (5)|w-\</w\> (5)|
|**e-s (8)**|o-w (7)|w-e (7)|
|e-w (5)|r-\</w\> (2)|w-i (3)|
|i-d (3)|s-t (8)||

4. Combinar el par más frecuente en el corpus.

||Corpus||
|-|-|-|
|l o w \</w\>|l o w e r \</w\>|n e w **es** t \</w\>|
|l o w \</w\>|l o w e r\</w\>|n e w **es** t \</w\>|
|l o w \</w\>|w i d **es** t \</w\>|n e w **es** t \</w\>|
|l o w \</w\>|w i d **es** t \</w\>|n e w **es** t \</w\>|
|l o w \</w\>|w i d **es** t \</w\>|n e w **es** t \</w\>|

5. Guarda la mejor pareja para el vocabulario.

|||||||||||
|-|-|-|-|-|-|-|-|-|-|
|d|e|i|l|n|o|r|s|t|w|
|**es**|-|-|-|-|-|-|-|-|-|

**Iteración 2:**

3. Calcule la frecuencia de un par de caracteres o secuencias de caracteres en corpus.

||||
|-|-|-|
|d-es (3)|l-o (7)|w-\</w\> (5)|
|e-r (2)|n-e (5)|w-es (5)|
|e-w (5)|o-w (7)|w-e (7)|
|**es-t (8)**|r-\</w\> (2)|w-i (3)|
|i-d (3)|t-\</w\> (8)||

4. Combinar el par más frecuente en el corpus.

||Corpus||
|-|-|-|
|l o w \</w\>|l o w e r \</w\>|n e w **est**  \</w\>|
|l o w \</w\>|l o w e r\</w\>|n e w **est**  \</w\>|
|l o w \</w\>|w i d **est**  \</w\>|n e w **est**  \</w\>|
|l o w \</w\>|w i d **est**  \</w\>|n e w **est** \</w\>|
|l o w \</w\>|w i d **est**  \</w\>|n e w **est**  \</w\>|

5. Guarda la mejor pareja para el vocabulario.

|||||||||||
|-|-|-|-|-|-|-|-|-|-|
|d|e|i|l|n|o|r|s|t|w|
|**es**|**est**|-|-|-|-|-|-|-|-|

**Después de 10 iteraciones, obtenemos:**

|||||||||||
|-|-|-|-|-|-|-|-|-|-|
|d|e|i|l|n|o|r|s|t|w|
|**es**|**est**|**est\</w\>**|**lo**|**low**|**low\</w\>**|**ne**|**new**|**newest\</w\>**|-|

En el momento de test, las palabras **OOV** se divide en secuencias de caracteres. 

Luego, las operaciones aprendidas se aplican para fusionar los caracteres en símbolos conocidos más grandes.

Para mayor información, se puede referir a [Neural Machine Translation of Rare Words with Subword Units](https://arxiv.org/pdf/1508.07909.pdf), Senrich R, et al., 2016.

Aquí hay un procedimiento paso a paso para representar palabras OOV:

1. Divide la palabra OOV en caracteres después de agregar \</w\>
1. Calcular un par de caracteres o secuencias de caracteres en una palabra
1. Seleccionar los pares presentes en las operaciones aprendidas
1. Combinar el par más frecuente
1. Repita los pasos 2 y 3 hasta que sea posible la fusión

#### <span style="color:blue">Implementación</span>

In [145]:
#importing library
import pandas as pd

#reading .txt file
text = pd.read_csv("../Datos/sample.txt")

#converting a dataframe into a single list 
corpus=[]
for row in text.values:
    # divide line text by spaces
    tokens = row[0].split(" ")
    for token in tokens:
        # append each token found
        corpus.append(token)

In [146]:
text

,Textos
0,low lower newest
1,low lower newest
2,low widest newest
3,low widest newest
4,low widest newest


In [147]:
corpus

['low',
 'lower',
 'newest',
 'low',
 'lower',
 'newest',
 'low',
 'widest',
 'newest',
 'low',
 'widest',
 'newest',
 'low',
 'widest',
 'newest']

In [148]:
#initlialize the vocabulary
vocab = list(set(" ".join(corpus)))
vocab.remove(' ')

#split the word into characters
corpus = [" ".join(token) for token in corpus]

#appending </w>
corpus=[token+' </w>' for token in corpus]

In [149]:
vocab

['w', 'o', 'e', 'l', 'r', 's', 't', 'i', 'd', 'n']

In [150]:
corpus

['l o w </w>',
 'l o w e r </w>',
 'n e w e s t </w>',
 'l o w </w>',
 'l o w e r </w>',
 'n e w e s t </w>',
 'l o w </w>',
 'w i d e s t </w>',
 'n e w e s t </w>',
 'l o w </w>',
 'w i d e s t </w>',
 'n e w e s t </w>',
 'l o w </w>',
 'w i d e s t </w>',
 'n e w e s t </w>']

In [151]:
import collections

#returns frequency of each word
corpus = collections.Counter(corpus)

#convert counter object to dictionary
corpus = dict(corpus)
print("Corpus:",corpus)

Corpus: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w e s t </w>': 5, 'w i d e s t </w>': 3}


La función a continuación cuenta la frecuencia de los bigramas de caracteres de cada palabra en el corpus:

In [152]:
#computer frequency of a pair of characters or character sequences
#accepts corpus and return frequency of each pair
def get_stats(corpus):
    pairs = collections.defaultdict(int)
    for word, freq in corpus.items():
        # divide string into characters
        symbols = word.split()
        # count frequencies of pairs
        for i in range(len(symbols)-1):
            # look the pairs and count
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

Finalmente, combinamos los pares más frecuentes:

In [153]:
#merges the most frequent pair in the corpus
#accepts the corpus and best pair
#returns the modified corpus 
import re
def merge_vocab(pair, corpus_in):
    corpus_out = {}
    # Ignore special characters
    bigram = re.escape(' '.join(pair))
    # Merge the sequence into the corpus
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    
    for word in corpus_in:
        w_out = p.sub(''.join(pair), word)
        corpus_out[w_out] = corpus_in[word]
    
    return corpus_out

In [154]:
#compute frequency of bigrams in a corpus
pairs = get_stats(corpus)
pairs

defaultdict(int,
            {('l', 'o'): 7,
             ('o', 'w'): 7,
             ('w', '</w>'): 5,
             ('w', 'e'): 7,
             ('e', 'r'): 2,
             ('r', '</w>'): 2,
             ('n', 'e'): 5,
             ('e', 'w'): 5,
             ('e', 's'): 8,
             ('s', 't'): 8,
             ('t', '</w>'): 8,
             ('w', 'i'): 3,
             ('i', 'd'): 3,
             ('d', 'e'): 3})

In [155]:
#compute the best pair
best = max(pairs, key=pairs.get)
print("Most Frequent pair:",best)

Most Frequent pair: ('e', 's')


In [156]:
#merge the frequent pair in corpus
corpus = merge_vocab(best, corpus)
print("After Merging:", corpus)

#convert a tuple to a string
best = "".join(list(best))

#append to merge list and vocabulary
merges = []
merges.append(best)
vocab.append(best)

After Merging: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w es t </w>': 5, 'w i d es t </w>': 3}


In [157]:
num_merges = 10
for i in range(num_merges):
    
    #compute frequency of bigrams in a corpus
    pairs = get_stats(corpus)
    
    #compute the best pair
    best = max(pairs, key=pairs.get)
    
    #merge the frequent pair in corpus
    corpus = merge_vocab(best, corpus)
    
    #append to merge list and vocabulary
    merges.append(best)
    vocab.append(best)

#convert a tuple to a string
merges_in_string = ["".join(list(i)) for i in merges]
print("BPE Merge Operations:",merges_in_string)

BPE Merge Operations: ['es', 'est', 'est</w>', 'lo', 'low', 'low</w>', 'ne', 'new', 'newest</w>', 'wi', 'wid']


In [158]:
vocab

['w',
 'o',
 'e',
 'l',
 'r',
 's',
 't',
 'i',
 'd',
 'n',
 'es',
 ('es', 't'),
 ('est', '</w>'),
 ('l', 'o'),
 ('lo', 'w'),
 ('low', '</w>'),
 ('n', 'e'),
 ('ne', 'w'),
 ('new', 'est</w>'),
 ('w', 'i'),
 ('wi', 'd')]

In [177]:
#applying BPE to OOV
oov ='lowest'

#tokenize OOV into characters
oov = " ".join(list(oov))

#append </w> 
oov = oov + ' </w>'

#create a dictionary
oov = { oov : 1}

In [178]:
oov

{'l o w e s t </w>': 1}

In [179]:
i=0
while(True):

    #compute frequency
    pairs = get_stats(oov)

    #extract keys
    pairs = pairs.keys()
    
    #find the pairs available in the learned operations
    ind=[merges.index(i) for i in pairs if i in merges]
    #print(ind)
    if(len(ind)==0):
        print("\nBPE Completed...")
        break
    
    #choose the most frequent learned operation
    best = merges[min(ind)]
    
    #merge the best pair
    oov = merge_vocab(best, oov)
    
    print("Iteration ",i+1, list(oov.keys())[0])
    i=i+1

[3]
Iteration  1 lo w e s t </w>
[4]
Iteration  2 low e s t </w>
[]

BPE Completed...


In [175]:
pairs = get_stats(oov)
pairs = pairs.keys()
print(pairs)
ind=[merges.index(i) for i in pairs if i in merges]
ind

dict_keys([('l', 'o'), ('o', 'w'), ('w', 'e'), ('e', 's'), ('s', 't'), ('t', '</w>')])


[3]

In [176]:
merges

['es',
 ('es', 't'),
 ('est', '</w>'),
 ('l', 'o'),
 ('lo', 'w'),
 ('low', '</w>'),
 ('n', 'e'),
 ('ne', 'w'),
 ('new', 'est</w>'),
 ('w', 'i'),
 ('wi', 'd')]

Modelos como GPT-2, RoBERTa, XLM y FlauBERT usan BPE para su tokenización.

En las siguientes lecciones veremos estos tipos de modelos a profundidad (Decoder-Only).

### <span style="color:blue">WordPiece</span>

Modelos como BERT, DistilBERT, Electra usan este tipo de tokenización.

Actualmente, hay dos implementaciones del algoritmo de WordPiece: de abajo hacia arriba y de arriba hacia abajo. 

El enfoque ascendente original se basa en BPE. 

BERT utiliza la implementación de arriba hacia abajo de WordPiece.

Sus comienzos datan de [Japanese and Korean Voice Search (Schuster et al., 2012)](https://static.googleusercontent.com/media/research.google.com/ja//pubs/archive/37842.pdf), y ganó popularidad gracias a BERT.

Recordemos, BPE toma un par de tokens (bytes), observa la frecuencia de cada par y fusiona el par que tiene la frecuencia combinada más alta. 

El proceso es codicioso (greedy), ya que busca la frecuencia combinada más alta en cada paso.

Entonces, ¿cuál es el problema con BPE? 🤔 

Puede tener instancias en las que hay más de una forma de codificar una palabra en particular. 

Entonces se vuelve difícil para el algoritmo elegir tokens de subpalabras ya que no hay forma de priorizar cuál usar primero. 

Por lo tanto, la misma entrada puede representarse mediante diferentes codificaciones que afectan la precisión de las representaciones aprendidas.

Miremos el siguiente vocabulario para un corpus pequeño:

|Número|Token|Frecuencia|
|-|-|-|
|1|li|3|
|2|l|5|
|3|ea|2|
|4|eb|4|
|5|near|6|
|6|bra|2|
|7|al|4|
|8|n|5|
|9|r|1|
|10|ge|11|
|11|g|7|
|12|e|8|

SI queremos tokenizar la frase **linear algebra**, podemos hacer lo siguiente:

linear = **li**+**near** ó **li** + **n** + **ea** + **r**

algebra = **al** + **ge** + **bra** ó **al** + **g** + **e** + **bra**

Como vemos, tenemos cuatro formas de poder codificar la frase, lo cual representa un problema para los modelos.

En la ciencia de datos, cuando avanzamos un paso por delante de la frecuencia o el conteo, buscamos enfoques probabilísticos. 

En WordPiece, en realidad hacemos lo mismo. La única diferencia entre WordPiece y BPE es la forma en que se agregan los pares de símbolos al vocabulario. 

En cada paso iterativo, WordPiece elige un par de símbolos que resultará en el mayor aumento de probabilidad al fusionarse. 

Maximizar la probabilidad de los datos de entrenamiento es equivalente a encontrar el par de símbolos cuya probabilidad dividida por la probabilidad del primero seguido por la probabilidad del segundo símbolo en el par es mayor que cualquier otro par de símbolos.

Por ejemplo, el algoritmo verificará si la probabilidad de ocurrencia de “es” es mayor que la probabilidad de ocurrencia de “e” seguida de “s”. 

La fusión ocurrirá solo si la probabilidad de "es" dividida por "e", "s" es mayor que cualquier otro par de símbolos.

Así, podemos decir que WordPiece evalúa lo que perderá fusionando los dos símbolos para asegurarse de que el paso que está dando realmente vale la pena o no.

Podemos resumir el algoritmo en los siguientes pasos, según el paper:

1. Inicialice el inventario de unidades de palabras con los caracteres base.
1. Construya un modelo de lenguaje sobre los datos de entrenamiento usando la palabra inventario de 1.
1. Genere una nueva unidad de palabras combinando dos unidades del inventario de palabras actual. El inventario de unidades de palabras se incrementará en 1 después de agregar esta nueva unidad de palabras. La nueva unidad de palabra se elige entre todas las posibles para que aumente al máximo la probabilidad de que los datos de entrenamiento se agreguen al modelo.
1. Vaya a 2 hasta que se alcance un límite predefinido de unidades de palabras o el aumento de probabilidad caiga por debajo de un cierto umbral.

Debe estar pensando que el entrenamiento debe ser un procedimiento computacionalmente costoso si se realiza utilizando la fuerza bruta. 

Si es así, entonces tienes razón.

La complejidad del tiempo es O(K²) donde K es el número de unidades de palabras actuales. 

En cuanto a cada iteración, necesitamos probar todas las combinaciones de pares posibles y entrenar un nuevo modelo de lenguaje cada vez. 

Sin embargo, el algoritmo de entrenamiento puede acelerarse significativamente siguiendo algunos trucos simples, como se explica en el artículo.

Podemos probar solo los pares que realmente existen en los datos de entrenamiento, probar solo los pares con una probabilidad significativa de ser los mejores (los que tienen altas prioridades), combinar varios pasos de agrupación en una sola iteración (posible para un grupo de pares que no se afectan entre sí). 

Según el documento, estas aceleraciones codiciosas ayudaron a construir un vocabulario de 200k para los conjuntos de datos de japonés y coreano en solo unas pocas horas en una sola máquina.

La implementación de este algoritmo se parece mucho entonces a la de BPE con unos pequeños cambios de optimización.

Así pues, Tensorflow y HuggingFace han creade excelentes API para no hacer el entrenamiento desde cero.

Veremos más adelante un ejemplo en concreto.

Si tiene curiosidad inmediata, puede mirar en [Subword tokenizers](https://www.tensorflow.org/text/guide/subwords_tokenizer) de Tensorflow.

### <span style="color:blue">Unigram</span>

El algoritmo Unigram se usa a menudo en SentencePiece, que es el algoritmo de tokenización que usan modelos como AlBERT, T5, mBART, Big Bird y XLNet.

En comparación con BPE y WordPiece, Unigram funciona en la otra dirección: comienza con un gran vocabulario y le quita tokens hasta que alcanza el tamaño de vocabulario deseado. 

Hay varias opciones para usar para construir ese vocabulario base: podemos tomar las subcadenas más comunes en palabras previamente tokenizadas, por ejemplo, o aplicar BPE en el corpus inicial con un tamaño de vocabulario grande.

En cada paso del entrenamiento, el algoritmo Unigram calcula una pérdida sobre el corpus dado el vocabulario actual. Luego, para cada símbolo en el vocabulario, el algoritmo calcula cuánto aumentaría la pérdida total si se eliminara el símbolo, y busca los símbolos que la aumentarían menos. 

Esos símbolos tienen un efecto menor en la pérdida general del corpus, por lo que, en cierto sentido, son "menos necesarios" y son los mejores candidatos para su eliminación.

Toda esta es una operación muy costosa, por lo que no solo eliminamos el símbolo único asociado con el aumento de pérdida más bajo, sino el porcentaje p (siendo p un hiperparámetro que puede controlar, generalmente 10 o 20) de los símbolos asociados con el aumento más bajo. aumento de pérdidas.

Este proceso luego se repite hasta que el vocabulario ha alcanzado el tamaño deseado.

Tenga en cuenta que nunca eliminamos los caracteres base, para asegurarnos de que cualquier palabra pueda tokenizarse.

Ahora, esto todavía es un poco vago: la parte principal del algoritmo es calcular una pérdida sobre el corpus y ver cómo cambia cuando eliminamos algunos tokens del vocabulario, pero aún no hemos explicado cómo hacerlo. 

Este paso se basa en el algoritmo de tokenización de un modelo de Unigram, por lo que nos sumergiremos en esto a continuación.

Para mostrar un ejemplo, vayamos a [Tokenization algorithm](https://huggingface.co/course/chapter6/7?fw=pt#tokenization-algorithm), de HuggingFace.

### <span style="color:blue">SentencePiece</span>

Sorprendentemente, en realidad no es un tokenizador.

En realidad, es un método para seleccionar tokens de una lista precompilada, optimizando el proceso de tokenización en función de un corpus proporcionado. SentencePiece, es el nombre de un paquete (disponible en [google/sentencepiece](https://github.com/google/sentencepiece/)) que implementa el algoritmo de regularización de subpalabras (todo por el mismo autor, Kudo, Taku). 

**Puntos fuertes de SentencePiece:**

1. Está implementado en C++ y es increíblemente rápido. Puede entrenar un tokenizador en un corpus de 10⁵ caracteres en segundos.
1. También es increíblemente rápido para tokenizar. Esto significa que puede usarlo directamente en datos de texto sin procesar, sin la necesidad de almacenar sus datos tokenizados en el disco.
1. La regularización de subpalabras es como una versión de texto del aumento de datos y puede mejorar en gran medida la calidad de su modelo.
1. Es independiente del espacio en blanco. Puede entrenar idiomas que no estén delimitados por espacios en blanco, como el chino y el japonés, con la misma facilidad que lo haría con el inglés o el francés.
1. Puede funcionar a nivel de byte, por lo que **casi** nunca necesitará usar tokens [UNK] o [OOV]. Esto no es específico solo de SentencePiece.

Está fundamentado en [Byte Pair Encoding is Suboptimal for Language Model Pretraining](https://arxiv.org/pdf/2004.03720.pdf).

En pocas palabras, SentencePiece usa una estrategia entre BPE y modelos unigram para aprovechar las múltiples formas que tienen las parejas de unirse como representación de una secuencia de símbolos.

A partir de estos métodos se optimiza una función que maximiza la probabilidad del conjunto de entrenamiento.

Para mayores detalles, podemos ir a [SentencePiece Tokenizer Demystified](https://towardsdatascience.com/sentencepiece-tokenizer-demystified-d0a3aac19b15), por Jonathan Kernes.

## <span style="color:blue">Datasets</span>

### <span style="color:blue">Desde el Hub de HF</span>

Haremos un ejercicio de tokenización.

Usemos el dataset Oscar para nuestros fines y la versión en latín, pues los otros idiomas tienen muchísima información.

In [ ]:
from datasets import load_dataset

In [2]:
dataset = load_dataset('oscar', 'unshuffled_deduplicated_la')

Generating train split:   0%|          | 0/18808 [00:00<?, ? examples/s]

Dataset oscar downloaded and prepared to /Users/moury/.cache/huggingface/datasets/oscar/unshuffled_deduplicated_la/1.0.0/84838bd49d2295f62008383b05620571535451d84545037bb94d6f3501651df2. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text'],
        num_rows: 18808
    })
})

In [9]:
dataset['train'][0]

{'id': 0,
 'text': 'Hæ sunt generationes Noë: Noë vir justus atque perfectus fuit in generationibus suis; cum Deo ambulavit.\nEcce ego adducam aquas diluvii super terram, ut interficiam omnem carnem, in qua spiritus vitæ est subter cælum: universa quæ in terra sunt, consumentur.\nTolles igitur tecum ex omnibus escis, quæ mandi possunt, et comportabis apud te: et erunt tam tibi, quam illis in cibum.'}

In [13]:
from tqdm.auto import tqdm

text_data = []
file_count = 0

for sample in tqdm(dataset['train']):
    sample = sample['text'].replace('\n', '')
    text_data.append(sample)
    if len(text_data) == 1_000:
        # once we git the 10K mark, save to file
        with open(f'./data/text/oscar_la/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1
# after saving in 10K chunks, we will have ~2082 leftover samples, we save those now too
with open(f'./data/text/oscar_la/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))

  0%|          | 0/18808 [00:00<?, ?it/s]

## Tokenizer 

In [18]:
from glob import glob

rutas = glob('./data/*/*/*.txt')
print(rutas)

['./data/text/oscar_la/text_10.txt', './data/text/oscar_la/text_11.txt', './data/text/oscar_la/text_13.txt', './data/text/oscar_la/text_12.txt', './data/text/oscar_la/text_16.txt', './data/text/oscar_la/text_17.txt', './data/text/oscar_la/text_15.txt', './data/text/oscar_la/text_14.txt', './data/text/oscar_la/text_2.txt', './data/text/oscar_la/text_3.txt', './data/text/oscar_la/text_1.txt', './data/text/oscar_la/text_0.txt', './data/text/oscar_la/text_4.txt', './data/text/oscar_la/text_5.txt', './data/text/oscar_la/text_7.txt', './data/text/oscar_la/text_6.txt', './data/text/oscar_la/text_8.txt', './data/text/oscar_la/text_9.txt', './data/text/oscar_la/text_18.txt']


In [2]:
from tokenizers import BertWordPieceTokenizer

In [3]:
tokenizer = BertWordPieceTokenizer()

In [ ]:
tokenizer.train(files=rutas,vocab_size=30_522,min_frequency=2,
                special_tokens=[
                    '<s>','<pad>','</s>','<unk>','<mask>'
                ])

In [85]:
import os
os.makedirs('latinberto',exist_ok=True)

In [86]:
tokenizer.save_model('latinberto')

['latinberto/vocab.json', 'latinberto/merges.txt']

In [89]:
from transformers import RobertaTokenizerFast

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained('./latinberto')

In [110]:
tokenizer.tokenize('et timenti bus')

['et', 'Ġtim', 'enti', 'Ġb', 'us']

In [111]:
tokenizer('et timenti bus')

{'input_ids': [0, 304, 1489, 1077, 386, 268, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [93]:
tokenizer.convert_ids_to_tokens([0, 304, 1489, 1077, 386, 268, 2])

['<s>', 'et', 'Ġtim', 'enti', 'Ġb', 'us', '</s>']

In [94]:
!wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
--2022-04-04 19:01:27--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.24.222
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.24.222|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191984949 (183M) [application/zip]
Saving to: ‘wikitext-103-raw-v1.zip’

wikitext-103-raw-v1 100%[===================>] 183.09M   175KB/s    in 12m 43s 

2022-04-04 19:14:12 (246 KB/s) - ‘wikitext-103-raw-v1.zip’ saved [191984949/191984949]



In [95]:
!unzip wikitext-103-raw-v1.zip

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Archive:  wikitext-103-raw-v1.zip
   creating: wikitext-103-raw/
  inflating: wikitext-103-raw/wiki.test.raw  
  inflating: wikitext-103-raw/wiki.valid.raw  
  inflating: wikitext-103-raw/wiki.train.raw  


In [96]:
from tokenizers import Tokenizer
from tokenizers.models import BPE

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

In [97]:
from tokenizers.trainers import BpeTrainer

trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

In [98]:
from tokenizers.pre_tokenizers import Whitespace

tokenizer.pre_tokenizer = Whitespace()

In [99]:
files = [f"wikitext-103-raw/wiki.{split}.raw" for split in ["test", "train", "valid"]]
tokenizer.train(files, trainer)

In [100]:
tokenizer.save("data/tokenizer-wiki.json")

### <span style="color:blue">Desde el computador local</span>

In [180]:
!wget https://github.com/crux82/squad-it/raw/master/SQuAD_it-train.json.gz
!wget https://github.com/crux82/squad-it/raw/master/SQuAD_it-test.json.gz

--2022-04-27 16:39:39--  https://github.com/crux82/squad-it/raw/master/SQuAD_it-train.json.gz
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/crux82/squad-it/master/SQuAD_it-train.json.gz [following]
--2022-04-27 16:39:40--  https://raw.githubusercontent.com/crux82/squad-it/master/SQuAD_it-train.json.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7725286 (7.4M) [application/octet-stream]
Saving to: ‘SQuAD_it-train.json.gz’

SQuAD_it-train.json 100%[===================>]   7.37M  2.39MB/s    in 3.1s    

2022-04-27 16:39:44 (2.39 MB/s) - ‘SQuAD_it-train.json.gz’ saved [7725286/7725286]

--202

In [181]:
!gzip -dkv SQuAD_it-*.json.gz

SQuAD_it-test.json.gz:	   87.4% -- replaced with SQuAD_it-test.json
SQuAD_it-train.json.gz:	   82.2% -- replaced with SQuAD_it-train.json


In [182]:
from datasets import load_dataset

squad_it_dataset = load_dataset("json", data_files="SQuAD_it-train.json", field="data")

Using custom data configuration default-c7f58bf9a4c25889


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /Users/moury/.cache/huggingface/datasets/json/default-c7f58bf9a4c25889/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [183]:
squad_it_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 442
    })
})

In [ ]:
squad_it_dataset["train"][0]

In [185]:
data_files = {"train": "SQuAD_it-train.json", "test": "SQuAD_it-test.json"}
squad_it_dataset = load_dataset("json", data_files=data_files, field="data")
squad_it_dataset

Using custom data configuration default-5d1738e3eece01bd


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /Users/moury/.cache/huggingface/datasets/json/default-5d1738e3eece01bd/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 442
    })
    test: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 48
    })
})

### <span style="color:blue">De forma remota</span>

In [ ]:
url = "https://github.com/crux82/squad-it/raw/master/"
data_files = {
    "train": url + "SQuAD_it-train.json.gz",
    "test": url + "SQuAD_it-test.json.gz",
}
squad_it_dataset = load_dataset("json", data_files=data_files, field="data")

### <span style="color:blue">Desde pandas (Back and Forth)</span>

In [186]:
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip"
!unzip drugsCom_raw.zip

from datasets import load_dataset

data_files = {"train": "drugsComTrain_raw.tsv", "test": "drugsComTest_raw.tsv"}
# \t is the tab character in Python
drug_dataset = load_dataset("csv", data_files=data_files, delimiter="\t")

drug_sample = drug_dataset["train"].shuffle(seed=42).select(range(1000))
# Peek at the first few examples
drug_sample[:3]


# CONVERTIR A PANDAS
drug_dataset.set_format("pandas")

drug_dataset["train"][:3]

--2022-04-27 16:49:38--  https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42989872 (41M) [application/x-httpd-php]
Saving to: ‘drugsCom_raw.zip’

drugsCom_raw.zip    100%[===================>]  41.00M  44.5KB/s    in 16m 30s 

2022-04-27 17:06:08 (42.4 KB/s) - ‘drugsCom_raw.zip’ saved [42989872/42989872]

Archive:  drugsCom_raw.zip
  inflating: drugsComTest_raw.tsv    
  inflating: drugsComTrain_raw.tsv   


Using custom data configuration default-8c0944eec81d0f16


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /Users/moury/.cache/huggingface/datasets/csv/default-8c0944eec81d0f16/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17


In [ ]:
# Crear datarame completo
train_df = drug_dataset["train"][:]

Desde aquí, podemos usar todas las operaciones de pandas que queramos y volver al dataset con las modificaciones:

In [ ]:
frequencies = (
    train_df["condition"]
    .value_counts()
    .to_frame()
    .reset_index()
    .rename(columns={"index": "condition", "condition": "frequency"})
)
frequencies.head()

In [ ]:
from datasets import Dataset

freq_dataset = Dataset.from_pandas(frequencies)
freq_dataset

### <span style="color:blue">Guardar en Disco y volver a cargar</span>

In [ ]:
freq_dataset.save_to_disk("freq_dataset")

In [ ]:
from datasets import load_from_disk

drug_dataset_reloaded = load_from_disk("freq_dataset")
drug_dataset_reloaded